In [10]:
pip install flask tensorflow requests pillow numpy

  Obtaining dependency information for flask from https://files.pythonhosted.org/packages/af/47/93213ee66ef8fae3b93b3e29206f6b251e65c97bd91d8e1c5596ef15af0a/flask-3.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for itsdangerous>=2.2 from https://files.pythonhosted.org/packages/04/96/92447566d16df59b2a776c0fb82dbc4d9e07cd95062562af01e408583fc4/itsdangerous-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for click>=8.1.3 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for blinker>=1.9 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/103.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/103.0 kB ? eta -:--:--
   ----------- --------------------------- 


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\Ayush\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, request, jsonify
import requests
from io import BytesIO
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

app = Flask(__name__)

# Load the model
model = tf.keras.models.load_model('verification_model.h5')

# Class labels
class_labels = ['electricity', 'garbage', 'road', 'unknown', 'water logging']

def load_and_preprocess_image_from_url(url):
    """Load and preprocess image from URL."""
    response = requests.get(url)
    img = image.load_img(BytesIO(response.content), target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize
    return img_array

def predict_image_class_from_url(url, threshold=0.7):
    """Predict image class from URL."""
    try:
        img_array = load_and_preprocess_image_from_url(url)
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions, axis=1)
        predicted_probability = np.max(predictions)
        
        if predicted_probability >= threshold:
            return {"class": class_labels[predicted_class_index[0]], "confidence": float(predicted_probability)}
        else:
            return {"class": "unknown", "confidence": float(predicted_probability)}
    except Exception as e:
        return {"error": str(e)}

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    """Endpoint to handle prediction requests."""
    try:
        if request.method == 'POST':
            # Get JSON data from the POST request
            data = request.json
            image_url = data.get("url")
        elif request.method == 'GET':
            # Get the URL parameter from the query string
            image_url = request.args.get("url")
        
        if not image_url:
            return jsonify({"error": "No URL provided"}), 400
        
        # Predict the class
        prediction = predict_image_class_from_url(image_url)
        return jsonify(prediction)
    except Exception as e:
        return jsonify({"error": str(e)}), 500


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.118.107:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 16:02:18] "GET /predict?https://www.shutterstock.com/image-photo/dhaka-bangladesh-may-22-2018-260nw-1096293797.jpg HTTP/1.1" 400 -
INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 16:02:40] "GET /predict?https://www.shutterstock.com/image-photo/dhaka-bangladesh-may-22-2018-260nw-1096293797.jpg HTTP/1.1" 400 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step


INFO:werkzeug:127.0.0.1 - - [15/Nov/2024 16:03:11] "GET /predict?url=https://www.shutterstock.com/image-photo/dhaka-bangladesh-may-22-2018-260nw-1096293797.jpg HTTP/1.1" 200 -


In [1]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
